# Les importations

In [ ]:
%pip install evaluate
%pip install sacrebleu rouge_score

In [ ]:
# Les importations
from transformers import get_linear_schedule_with_warmup, AutoTokenizer,TrainerCallback,AutoModelForCausalLM, DataCollatorForSeq2Seq,AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer,EarlyStoppingCallback
from datasets import load_dataset,Dataset, DatasetDict
import evaluate
import numpy as np
from torch.cuda.amp import autocast, GradScaler
import torch
import tensorflow as tf

### Chargement du dataset

In [ ]:
# notre fichier d'arabe
#with open("/kaggle/input/arb-fr-2/arabe_tchad3.txt", "r") as file:
with open("for_dataset_arb.txt", "r") as file:
    # On lit le contenu du fichier
    data_arb = file.readlines()
# notre fichier de français
#with open("/kaggle/input/arb-fr-2/V_francais_3.txt", "r") as file:
with open("for_dataset_fr.txt", "r") as file:
    # On lit le contenu du fichier
    data_fr = file.readlines()

# Supprimons les lignes vides
data_fr = [line for line in data_fr if line.strip()]
data_arb = [line for line in data_arb if line.strip()]
print("fr_lengh :", len(data_fr))
print("arb_lengh :", len(data_arb))


### Division en train, test, validation

In [ ]:
# Création du dictionnaire
dataset_dict = {"translation": [{"arb": a.strip(), "fr": f.strip()} for a, f in zip(data_arb, data_fr)]}

# Création du dataset
dataset = Dataset.from_dict(dataset_dict)
# On va diviser notre dataset en deux parties
 #train_dataset =  DatasetDict({"train":test['train']})

train_dataset = dataset.train_test_split(test_size=0.1)
# On entregistre notre dataset
#save_path = "dataset_arabe_fr"
#train_dataset.save_to_disk(save_path)
# On va charger notre dataset
#train_dataset = DatasetDict.load_from_disk(save_path)
#train_dataset

# L'entrainement pour la traduction de l'arabe vers le français

### La definition des langue cible et d'entrée

In [ ]:
def extract_langage(examples) :
    # On va extraire le langage arabe et le langage français
    inputs = [arb['arb'].strip() for arb in examples["translation"]] 
    target = [fr['fr'].strip()  for fr in examples["translation"]]
    return {'inputs' : inputs, 'target' : target}

# On va appliquer la fonction sur notre dataset
train_dataset = train_dataset.map(extract_langage, batched=True, remove_columns=["translation"])
print(train_dataset["train"]["target"][0])
print(train_dataset['train']["inputs"][0])




In [ ]:
from huggingface_hub import notebook_login

notebook_login()

### Evaluation du model

In [ ]:
# Charger les métriques
bleu_metric = evaluate.load("sacrebleu")
rouge_metric = evaluate.load("rouge")
meteor_metric = evaluate.load("meteor")
#accuracy_metric = evaluate.load("accuracy")  # Pour évaluer la précision


### La definition des metrics

In [ ]:
def postprocess_text(preds, labels):
    preds = [pred.strip() for pred in preds]
    labels = [[label.strip()] for label in labels]  # ROUGE/METEOR attend des listes imbriquées
    return preds, labels

def compute_metrics(eval_preds):
    preds, labels = eval_preds
    if isinstance(preds, tuple):
        preds = preds[0]

    # Décodage des prédictions et labels
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Nettoyage du texte
    decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)

    # 🏆 Calcul des métriques
    bleu_result = bleu_metric.compute(predictions=decoded_preds, references=decoded_labels)
    rouge_result = rouge_metric.compute(predictions=decoded_preds, references=decoded_labels)
    meteor_result = meteor_metric.compute(predictions=decoded_preds, references=decoded_labels)
    # accuracy_result = accuracy_metric.compute(predictions=decoded_preds, references=decoded_labels) # Removed accuracy metric

    # 📊 Récupération des scores
    result = {
        "bleu": bleu_result["score"],
        "rouge": rouge_result["rougeL"],  # Utilisation du score ROUGE-L
        "meteor": meteor_result["meteor"],
        # "accuracy": accuracy_result["accuracy"]  # Précision des prédictions # Removed accuracy metric
    }

    # Calcul de la longueur moyenne des prédictions
    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in preds]
    result["gen_len"] = np.mean(prediction_lens)

    # Arrondir les scores
    result = {k: round(v, 4) for k, v in result.items()}
    return result

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Using device: {device}')

In [ ]:
import os
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"


In [ ]:
early =  EarlyStoppingCallback(early_stopping_patience = 5)
#early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

class MemoryCleanupCallback(TrainerCallback):
    def on_step_end(self, args, state, control, **kwargs):
        torch.cuda.empty_cache()  # Libérer la mémoire GPU inutilisée


class AMPCallback(TrainerCallback):
    def __init__(self):  #torch.amp.GradScaler('cuda', args...)
        super().__init__()
        self.scaler = GradScaler('cuda')  # Initialise le scaler pour AMP

    def on_step_begin(self, args, state, control, **kwargs):
        control.should_log = True  # Forcer l'affichage des logs

    def on_backward_end(self, args, state, control, **kwargs):
        with autocast():  # Active AMP pour les calculs
            self.scaler.scale(state.loss).backward()
            self.scaler.step(state.optimizer)
            self.scaler.update()


In [ ]:
checkpoint = "facebook/nllb-200-distilled-600M"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = AutoModelForSeq2SeqLM.from_pretrained(checkpoint, token = os.environ.get('TOKEN'))

# Appliquons la tokenisation sur notre dataset
max_input_length = 128
max_target_length = 128

# Dans votre code pour Arabe Tchadien (Latin) -> Français
SOURCE_LANG_CODE_NLLB = "arb_Latn" 
TARGET_LANG_CODE_NLLB = "fra_Latn"

def preprocess_function(examples):
    tokenizer.src_lang = SOURCE_LANG_CODE_NLLB
    model_inputs = tokenizer(
        examples["inputs"],
        max_length=max_input_length,
        truncation=True
    )

    # Nettoyer les cibles : enlever None ou chaînes vides
    clean_targets = []
    for t in examples["target"]:
        if isinstance(t, str) and t.strip():
            clean_targets.append(t)
        else:
            clean_targets.append(" ")

    tokenizer.tgt_lang = TARGET_LANG_CODE_NLLB
    labels = tokenizer(
        clean_targets,
        max_length=max_target_length,
        truncation=True,
        text_target=clean_targets  # Pour compatibilité future
    )

    model_inputs["labels"] = labels["input_ids"]
    
    return model_inputs
# On va appliquer la fonction sur notre dataset
train_dataset = train_dataset.map(preprocess_function, batched=True, remove_columns=["inputs", "target"])

# On va maintenant definir notre collator
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)

In [ ]:
training_args = Seq2SeqTrainingArguments(
    output_dir="/my_train_model_facebook_nllb_200",
    #run_name="nllb_experiment_v1",   Nom de run distinct
    report_to="none",
    hub_model_id= os.environ.get('HUB_ID'),  # ✅ Ajoute ton Repo ID
    eval_strategy="steps",
    learning_rate=2e-5,
    #learning_rate=1e-5,
    gradient_checkpointing=True,  # ✅ Active la réduction mémoire
    #use_cache = False,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=50,
    metric_for_best_model="eval_loss",
    predict_with_generate=True,
    fp16=True, #change to bf16=True #for XPU
    push_to_hub=True,
    remove_unused_columns=False,
    load_best_model_at_end=True,
    save_strategy="steps",
    #save_steps = 500,
    #resume_from_checkpoint=True,
    gradient_accumulation_steps=4,

)



"""class CheckpointCleanupCallback(TrainerCallback):
    def on_save(self, args, state, control, **kwargs):
        clean_old_checkpoints(args.output_dir, keep=10)  # ✅ Supprime les anciens checkpoints """

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset["train"],
    eval_dataset=train_dataset["test"],
    processing_class=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    callbacks=[early, AMPCallback(),MemoryCleanupCallback() ], #CheckpointCleanupCallback()
)
torch.cuda.empty_cache()  # Libérer la mémoire GPU inutilisée
trainer.train()

In [ ]:
trainer.push_to_hub()

In [ ]:
# On va maintenant evaluer notre modèle
trainer.evaluate()

### Telechargement de mon modele du hub et enfin l'evaluer

### Testons notre modele

In [ ]:
# on fait un test en traduisant une phrase
text_to_translate = "ju sawani."  # A Remplacer par la phrase que vous voulez traduire

# --- 1. Définir les codes de langue NLLB utilisés lors de l'entraînement ---
# Pour l'arabe tchadien en script latin, votre tokenizer par défaut utilisait "eng_Latn" comme src_lang.
SOURCE_LANG_CODE_NLLB = "arb_Latn" # C'est le code proxy que votre modèle a appris à associer à l'arabe tchadien (latin)
TARGET_LANG_CODE_NLLB = "fra_Latn" # Le code NLLB pour le français

# --- 2. Votre phrase à traduire ---
#text_to_translate = "Inti maalak?"  # Phrase en arabe tchadien (latin)

# --- 3. Charger le tokenizer et le modèle ---
model_name = os.environ.get('HUB_ID')  # Votre modèle affiné

# Assurez-vous que c'est bien la version "fast" qui est chargée si c'est ce qui est par défaut pour votre environnement
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

# --- 4. Configurer le tokenizer pour la langue source ---
# C'est crucial : indique au tokenizer d'ajouter le bon token de langue source.
tokenizer.src_lang = SOURCE_LANG_CODE_NLLB

# --- 5. Tokeniser l'entrée ---
# Le tokenizer ajoutera automatiquement le token __eng_Latn__ au début de la séquence.
inputs = tokenizer(text_to_translate, return_tensors="pt", padding=True, truncation=True)

# Pour la vérification (optionnel) :
print("Tokens d'entrée (IDs) :", inputs["input_ids"][0].tolist())
print("Tokens d'entrée décodés :", tokenizer.convert_ids_to_tokens(inputs["input_ids"][0]))

# --- 6. Générer la traduction ---
# Correction ici pour obtenir l'ID du token de langue cible
# On préfixe le code de langue NLLB avec "__" et on le suffixe avec "__"
# pour qu'il corresponde à la forme des tokens de langue NLLB (par exemple, "__fra_Latn__").
target_lang_token_id = tokenizer.convert_tokens_to_ids(f"__{TARGET_LANG_CODE_NLLB}__")

outputs = model.generate(
    **inputs,
    forced_bos_token_id=target_lang_token_id, # Utilisation de l'ID corrigé
    max_new_tokens=128 # Limitez la longueur de la traduction si nécessaire
)

# --- 7. Décoder la traduction ---
translation = tokenizer.batch_decode(outputs, skip_special_tokens=True)

print("\nLa traduction est :", translation[0])

# La traduction du Français vers l'arabe

### Chargement de notre dataset

In [ ]:
def extract_langage(examples) :
    # On va extraire le langage arabe et le langage français
    inputs = [fr['fr'] for fr in examples["translation"]]
    target = [arb['arb'] for arb in examples["translation"]]
    return {'inputs' : inputs, 'target' : target}

# On va appliquer la fonction sur notre dataset
train_dataset = train_dataset.map(extract_langage, batched=True, remove_columns=["translation"])
print(train_dataset["train"]["target"][0])
print(train_dataset['train']["inputs"][0])


### Pretraitement de notre dataset

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

### Evaluation du model

In [ ]:
# Charger les métriques
bleu_metric = evaluate.load("sacrebleu")
rouge_metric = evaluate.load("rouge")
meteor_metric = evaluate.load("meteor")
#accuracy_metric = evaluate.load("accuracy")  # Pour évaluer la précision


In [ ]:
def postprocess_text(preds, labels):
    preds = [pred.strip() for pred in preds]
    labels = [[label.strip()] for label in labels]  # ROUGE/METEOR attend des listes imbriquées
    return preds, labels

def compute_metrics(eval_preds):
    preds, labels = eval_preds
    if isinstance(preds, tuple):
        preds = preds[0]

    # Décodage des prédictions et labels
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Nettoyage du texte
    decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)

    # 🏆 Calcul des métriques
    bleu_result = bleu_metric.compute(predictions=decoded_preds, references=decoded_labels)
    rouge_result = rouge_metric.compute(predictions=decoded_preds, references=decoded_labels)
    meteor_result = meteor_metric.compute(predictions=decoded_preds, references=decoded_labels)
    # accuracy_result = accuracy_metric.compute(predictions=decoded_preds, references=decoded_labels) # Removed accuracy metric

    # 📊 Récupération des scores
    result = {
        "bleu": bleu_result["score"],
        "rouge": rouge_result["rougeL"],  # Utilisation du score ROUGE-L
        "meteor": meteor_result["meteor"],
        # "accuracy": accuracy_result["accuracy"]  # Précision des prédictions # Removed accuracy metric
    }

    # Calcul de la longueur moyenne des prédictions
    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in preds]
    result["gen_len"] = np.mean(prediction_lens)

    # Arrondir les scores
    result = {k: round(v, 4) for k, v in result.items()}
    return result

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Using device: {device}')

In [ ]:
import os
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"


In [ ]:
early =  EarlyStoppingCallback(early_stopping_patience = 5)
#early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

class MemoryCleanupCallback(TrainerCallback):
    def on_step_end(self, args, state, control, **kwargs):
        torch.cuda.empty_cache()  # Libérer la mémoire GPU inutilisée


class AMPCallback(TrainerCallback):
    def __init__(self):  #torch.amp.GradScaler('cuda', args...)
        super().__init__()
        self.scaler = GradScaler('cuda')  # Initialise le scaler pour AMP

    def on_step_begin(self, args, state, control, **kwargs):
        control.should_log = True  # Forcer l'affichage des logs

    def on_backward_end(self, args, state, control, **kwargs):
        with autocast():  # Active AMP pour les calculs
            self.scaler.scale(state.loss).backward()
            self.scaler.step(state.optimizer)
            self.scaler.update()


In [ ]:
checkpoint = "facebook/nllb-200-distilled-600M"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = AutoModelForSeq2SeqLM.from_pretrained(checkpoint, token = os.environ.get('TOKEN'))

# Appliquons la tokenisation sur notre dataset
max_input_length = 128
max_target_length = 128

# Dans votre code pour Arabe Tchadien (Latin) -> Français
SOURCE_LANG_CODE_NLLB = "fra_Latn"
TARGET_LANG_CODE_NLLB = "arb_Latn"

def preprocess_function(examples):
    tokenizer.src_lang = SOURCE_LANG_CODE_NLLB
    model_inputs = tokenizer(
        examples["inputs"],
        max_length=max_input_length,
        truncation=True
    )

    # Nettoyer les cibles : enlever None ou chaînes vides
    clean_targets = []
    for t in examples["target"]:
        if isinstance(t, str) and t.strip():
            clean_targets.append(t)
        else:
            clean_targets.append(" ")

    tokenizer.tgt_lang = TARGET_LANG_CODE_NLLB
    labels = tokenizer(
        clean_targets,
        max_length=max_target_length,
        truncation=True,
        text_target=clean_targets  # Pour compatibilité future
    )

    model_inputs["labels"] = labels["input_ids"]
    
    return model_inputs
# On va appliquer la fonction sur notre dataset
train_dataset = train_dataset.map(preprocess_function, batched=True, remove_columns=["inputs", "target"])

# On va maintenant definir notre collator
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)

In [ ]:
training_args = Seq2SeqTrainingArguments(
    output_dir="/my_train_model_facebook_nllb_200",
    #run_name="nllb_experiment_v1",   Nom de run distinct
    report_to="none",
    hub_model_id= os.environ.get('HUB_ID_2'),  # ✅ Ajoute ton Repo ID
    eval_strategy="steps",
    learning_rate=2e-5,
    #learning_rate=1e-5,
    gradient_checkpointing=True,  # ✅ Active la réduction mémoire
    #use_cache = False,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=50,
    metric_for_best_model="eval_loss",
    predict_with_generate=True,
    fp16=True, #change to bf16=True #for XPU
    push_to_hub=True,
    remove_unused_columns=False,
    load_best_model_at_end=True,
    save_strategy="steps",
    #save_steps = 500,
    #resume_from_checkpoint=True,
    gradient_accumulation_steps=4,

)



"""class CheckpointCleanupCallback(TrainerCallback):
    def on_save(self, args, state, control, **kwargs):
        clean_old_checkpoints(args.output_dir, keep=10)  # ✅ Supprime les anciens checkpoints """

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset["train"],
    eval_dataset=train_dataset["test"],
    processing_class=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    callbacks=[early, AMPCallback(),MemoryCleanupCallback() ], #CheckpointCleanupCallback()
)
torch.cuda.empty_cache()  # Libérer la mémoire GPU inutilisée
trainer.train()

In [ ]:
trainer.push_to_hub()

In [ ]:
# On va maintenant evaluer notre modèle
trainer.evaluate()

### Telechargement de mon modele du hub et enfin l'evaluer

### Testons notre modele

In [ ]:
# on fait un test en traduisant une phrase
text_to_translate = "Il m'était arrivé de m'égarer "  # A Remplacer par la phrase que vous voulez traduire
# on fait un test en traduisant une phrase

# --- 1. Définir les codes de langue NLLB utilisés lors de l'entraînement ---
# Pour l'arabe tchadien en script latin, votre tokenizer par défaut utilisait "eng_Latn" comme src_lang.
SOURCE_LANG_CODE_NLLB = "fra_Latn" # C'est le code proxy que votre modèle a appris à associer à l'arabe tchadien (latin)
TARGET_LANG_CODE_NLLB = "arb_Latn"# Le code NLLB pour le français

# --- 2. Votre phrase à traduire ---
#text_to_translate = "Inti maalak?"  # Phrase en arabe tchadien (latin)

# --- 3. Charger le tokenizer et le modèle ---
model_name = os.environ.get("HUB_ID_2")  # Remplacez par le nom de votre modèle 

# Assurez-vous que c'est bien la version "fast" qui est chargée si c'est ce qui est par défaut pour votre environnement
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

# --- 4. Configurer le tokenizer pour la langue source ---
# C'est crucial : indique au tokenizer d'ajouter le bon token de langue source.
tokenizer.src_lang = SOURCE_LANG_CODE_NLLB

# --- 5. Tokeniser l'entrée ---
# Le tokenizer ajoutera automatiquement le token __eng_Latn__ au début de la séquence.
inputs = tokenizer(text_to_translate, return_tensors="pt", padding=True, truncation=True)

# Pour la vérification (optionnel) :
print("Tokens d'entrée (IDs) :", inputs["input_ids"][0].tolist())
print("Tokens d'entrée décodés :", tokenizer.convert_ids_to_tokens(inputs["input_ids"][0]))

# --- 6. Générer la traduction ---
# Correction ici pour obtenir l'ID du token de langue cible
# On préfixe le code de langue NLLB avec "__" et on le suffixe avec "__"
# pour qu'il corresponde à la forme des tokens de langue NLLB (par exemple, "__fra_Latn__").
target_lang_token_id = tokenizer.convert_tokens_to_ids(f"__{TARGET_LANG_CODE_NLLB}__")
#print(target_lang_token_id)
outputs = model.generate(
    **inputs,
    forced_bos_token_id=target_lang_token_id, # Utilisation de l'ID corrigé
    max_new_tokens=256 # Limitez la longueur de la traduction si nécessaire
)

# --- 7. Décoder la traduction ---
translation = tokenizer.batch_decode(outputs, skip_special_tokens=True)

print("\nLa traduction est :", translation[0])